In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import altair as alt
from scipy.stats import pearsonr
import re

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [ ]:
counts_path = '../Data/ATG_lib_data/counts' #Path to counts for ATG library variants
x1a_annotations = '../Data/ATG_lib_data/X1A_annotations.xlsx' #Path to file with annotations for X1A variants

In [ ]:
def read_counts(path): #Reads all counts
    data_path = Path(counts_path) #Creates path object

    all_files = list(data_path.glob('*tsv')) #Creates list of all count TSV files

    counts_dict = {} #Empty dictionary to collect counts dataframes
    
    for path in all_files: #Iterates through all_files list and reads counts TSV files into dataframes and adds to counts_dict dictionary

        if 'RNA' in str(path): #Reads RNA counts
            df = pd.read_csv(path, sep = '\t')

            if 'RNA_R1R2R3' in str(path):
                counts_dict['RNA_D05_rep1'] = df
            elif 'RNA_R4R5R6' in str(path):
                counts_dict['RNA_D05_rep2'] = df
            elif 'RNA_R7R8R9' in str(path):
                counts_dict['RNA_D05_rep3'] = df
                
        else: #Reads DNA counts
            df = pd.read_csv(path, sep = '\t')
    
            str_path = str(path)
    
            if 'lib_counts' in str_path:
                counts_dict['lib'] = df
            elif 'NC' in str_path:
                counts_dict['NC'] = df
            elif 'R1R2R3_D05' in str_path:
                counts_dict['D05_rep1'] = df
            elif 'R4R5R6_D05' in str_path:
                counts_dict['D05_rep2'] = df
            elif 'R7R8R9_D05' in str_path:
                counts_dict['D05_rep3'] = df
            elif 'R3_D13' in str_path:
                counts_dict['D13_rep1'] = df
            elif 'R6_D13' in str_path:
                counts_dict['D13_rep2'] = df
            elif 'R9_D13' in str_path:
                counts_dict['D13_rep3'] = df

    return counts_dict

In [ ]:
def add_freq(dict): #Adds DNA/RNA frequency columns
    
    keys = list(dict.keys()) #Gets all keys from dictionary

    freq_dicts = {} #New dictionary to hold dataframes with frequency columns

    #Iterates through provided dict and adds frequency column
    for key in keys:
        if key != 'NC': #NC skipped
            df = dict[key] #Gets dataframe from dictionary

            total_count = df['Count'].sum() #Gets total number of variants
            df['freq'] = df['Count'] / total_count #Creates frequency column

            freq_dicts[key] = df #Adds to new dictionary

    keys = list(freq_dicts.keys()) #Gets list of keys for future use in downstream functions
    return freq_dicts,keys

In [ ]:
def mutate_snvs(dna_sequence): #Mutates all possible SNVs of provided DNA sequence
    snvs = []
    i = 0
    while i < len(dna_sequence):
        if dna_sequence[i] == "A":
            snvs.append(dna_sequence[:i] + "T" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "C" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "G" + dna_sequence[i + 1 :])
        elif dna_sequence[i] == "T":
            snvs.append(dna_sequence[:i] + "A" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "C" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "G" + dna_sequence[i + 1 :])
        elif dna_sequence[i] == "C":
            snvs.append(dna_sequence[:i] + "A" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "T" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "G" + dna_sequence[i + 1 :])
        else:
            snvs.append(dna_sequence[:i] + "A" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "T" + dna_sequence[i + 1 :])
            snvs.append(dna_sequence[:i] + "C" + dna_sequence[i + 1 :])
        i += 1
    return snvs

In [ ]:
def reverse_complement_string(seq_string): #Reverse complement and returns string
    reverse_seq = seq_string[::-1]
    reverse_comp_list = []
    for char in reverse_seq:
        if char == "A":
            reverse_comp_list.append("T")
        elif char == "G":
            reverse_comp_list.append("C")
        elif char == "C":
            reverse_comp_list.append("G")
        else:
            reverse_comp_list.append("A")
    reverse_compliment_str = "".join(reverse_comp_list)
    return reverse_compliment_str

In [ ]:
def compare_strings(str1, str2, coord_offset): #Special function to compare strings and return position with difference
    
    list_str1 = [] #List for string 1 characters
    list_str2 = [] #List for string 2 charaacters

    #Appends strings 1 and 2 to respective lists
    for char in str1:
        list_str1.append(char)

    for char in str2:
        list_str2.append(char)

    #Iterates and compares each element in str1 and 2 lists. 
    i = 0 
    while i < len(list_str1):
        if list_str1[i] == list_str2[i]:
            i += 1
        else:
            output_str = str(i + coord_offset) + ':' + list_str2[i] #If characters not the same, difference and position returned, coord_offset used for genomic coordinate
            
            return output_str
            
            i += 1

In [ ]:
def annotate_snv_lib(): #Annotates all SNVs in SNV lib with position IDs
    seq = 'CCATGGA' #original WT 7bp library around Met26

    coding_snvs = mutate_snvs(seq) #mutates all possible SNVs
    rev_coding_snvs = {} #Dictionary to hold SNVs that have been reverse complemented (used for pos_id merging)

    for snv in coding_snvs: #Iterates through all SNVs and reverse complements
        rev_coding_snvs[snv] = reverse_complement_string(snv)

    rev_seq = 'TCCATGG'

    mapped_snvs = {}
    for snv in coding_snvs: #iterates through SNVs and generates position IDs for merging
        pos_id = compare_strings(rev_seq, rev_coding_snvs[snv], 214809490)
        mapped_snvs[snv] = pos_id



    return mapped_snvs
    

In [ ]:
def annotate_vars(dict,keys, snv_map, annotations): #Annotates variants with Consequences and AA substitution

    annotation_df = pd.read_excel(annotations) #Reads annotations file
    annotated_dfs = {} #Diciotnary to hold annotated data frames
    
    for key in keys: #Annotates all dataframes
        df = dict[key] #Gets one dataframe
        df['canonical_start'] = df['Sequence'].transform(lambda x: x[15:18]) #Gets canonical start codon
        df['second_start'] = df['Sequence'].transform(lambda x: x[90:93]) #Gets second start codon
        df['snv_lib'] = df['Sequence'].transform(lambda x: x[88:95]) #Adds SNV lib sequence

        df['pos_id'] = df['snv_lib'].transform(lambda x: snv_map[x]) #Adds pos_id
        df['start_pos_id'] = df['canonical_start'] + ':' + df['pos_id'] #Adds combined column with canonical start codon and pos_id
        df = pd.merge(df, annotation_df, how = 'left', on = 'pos_id') #Merges

        df = df[['Count', 'freq', 'canonical_start', 'second_start', 'pos_id', 'pos', 'allele', 'AAsub', 'Consequence','start_pos_id']] #Gets important columns
        df = df.loc[:,['pos', 'allele', 'pos_id', 'Consequence', 'AAsub', 'canonical_start', 'second_start', 'Count', 'freq', 'start_pos_id']] #Reorders columns 

        new_count_name = key + '_count' #Name for replicate count column
        new_freq_name = key + '_freq' #Name for replicate frequency column

        df = df.rename(columns = {'Count': new_count_name, 'freq': new_freq_name}) #Columns for counts and frequency renamed
        
        annotated_dfs[key] = df #Annotated df added to dictionary

    return annotated_dfs

        
        

In [ ]:
def combine_dfs(dict, keys): #Merges all dataframes
    
    base_df = dict['lib'] #Base dataframe with column names and annotated variants
    base_df = base_df[['pos', 'allele', 'pos_id', 'Consequence', 'AAsub', 'canonical_start', 'second_start', 'start_pos_id']] #Key columns pulled

    for key in keys: #Merges all dataframes
        to_merge = dict[key] #Gets one dataframe for merging

        count_name = key + '_count' #Gets count column name
        freq_name = key + '_freq' #Gets freq column name
        to_merge = to_merge[['start_pos_id', count_name, freq_name]] #Gets columns for merging

        base_df = pd.merge(base_df, to_merge, how = 'left', on = 'start_pos_id') #Merges

    #RNA vs. DNA freq columsn used in RNA
    base_df['rep1_RNAovDNA'] = base_df['RNA_D05_rep1_freq'] / base_df['D05_rep1_freq']
    base_df['rep2_RNAovDNA'] = base_df['RNA_D05_rep2_freq'] / base_df['D05_rep2_freq']
    base_df['rep3_RNAovDNA'] = base_df['RNA_D05_rep3_freq'] / base_df['D05_rep3_freq']
    
    return base_df

In [ ]:
def qc_scatter(pair_list, df): #Function for QC scatter plot

    all_plots = []
    for pair in pair_list:
        rep1, rep2 = pair
        r, p_value = pearsonr(df[rep1], df[rep2])
        scatter = alt.Chart(df).mark_point().encode(
            x = rep1,
            y = rep2,
            tooltip = [alt.Tooltip('canonical_start', title = 'Canonical Start: '),
                       alt.Tooltip('second_start', title = 'Second Start: '),
                       alt.Tooltip('AAsub', title = 'AA Substitution')
                      ]
        ).properties(
            title = rep1 + ' vs. ' + rep2 + ' (r = ' + str(round(r,3)) + ')'
        ).interactive()

        trendline = scatter.transform_regression(rep1, rep2).mark_line(color = 'red')
        scatter = trendline + scatter

        all_plots.append(scatter)

    final_plot = alt.hconcat(all_plots[0], all_plots[1], all_plots[2])
    #final_plot.display()

    return final_plot

In [ ]:
def qc_plots(df): #Generates DNA replicate vs. replicate frequency correlation plots and RNA correlation plots
    
    d5_pairs = [('D05_rep1_freq', 'D05_rep2_freq'), ('D05_rep1_freq', 'D05_rep3_freq'), ('D05_rep2_freq', 'D05_rep3_freq')] #Pairs of D5 replicates to be tested
    d13_pairs = [('D13_rep1_freq', 'D13_rep2_freq'), ('D13_rep1_freq', 'D13_rep3_freq'), ('D13_rep2_freq', 'D13_rep3_freq')] #Pairs of D13 replicats to be tested

    d5_all = qc_scatter(d5_pairs, df) #Generates all D5 scatter plots
    
    d13_all = qc_scatter(d13_pairs, df) #Generates all D13 scatter plots
 
    full_corr_plot = d5_all & d13_all #Concatenates all plots 
    full_corr_plot.display() #Dispalys plot
    
    #Library QC plot with double stacked bars
    lib_plot = alt.Chart(df).mark_bar().encode(
        x = 'pos:O',
        y = 'lib_freq',
        color = 'canonical_start'
    ).properties(
        title = 'Replicate vs. Replicate DNA Freq.',
        width = 600,
        height = 400
    )

    lib_plot.display()


    #RNA QC plots
    d5_rna_pairs = [('RNA_D05_rep1_freq', 'RNA_D05_rep2_freq'), ('RNA_D05_rep1_freq', 'RNA_D05_rep3_freq'), ('RNA_D05_rep2_freq', 'RNA_D05_rep3_freq')] #Rep vs. rep D5 RNA frequency
    intra_rep_rna_dna_pairs = [('RNA_D05_rep1_freq', 'D05_rep1_freq'), ('RNA_D05_rep2_freq', 'D05_rep2_freq'), ('RNA_D05_rep3_freq', 'D05_rep3_freq')] #Rep vs. rep D5 RNA vs. DNA
    rnadna_ratios = [('rep1_RNAovDNA', 'rep2_RNAovDNA'), ('rep1_RNAovDNA', 'rep3_RNAovDNA'), ('rep2_RNAovDNA', 'rep3_RNAovDNA')] #Rep vs. rep RNA/DNA frequency

    #Builds all scatter plots
    rna_vs_rna = qc_scatter(d5_rna_pairs, df)
    intra_rep_rna_dna = qc_scatter(intra_rep_rna_dna_pairs, df)
    rnadna_ratios_qc = qc_scatter(rnadna_ratios, df)

    full_rna_plot = rna_vs_rna & intra_rep_rna_dna & rnadna_ratios_qc

    full_rna_plot.display()
    
    
    

In [ ]:
def dna_score(df): #Adds DNA scores
    original_df = df[['start_pos_id', 'lib_count', 'lib_freq',
       'D05_rep2_count', 'D05_rep2_freq', 'RNA_D05_rep3_count',
       'RNA_D05_rep3_freq', 'RNA_D05_rep1_count', 'RNA_D05_rep1_freq',
       'D13_rep2_count', 'D13_rep2_freq', 'D05_rep3_count', 'D05_rep3_freq',
       'RNA_D05_rep2_count', 'RNA_D05_rep2_freq', 'D13_rep1_count',
       'D13_rep1_freq', 'D05_rep1_count', 'D05_rep1_freq', 'D13_rep3_count',
       'D13_rep3_freq']]
    
    df = df.rename(columns = {'D05_rep1_count': 'D05_R1',
                              'D05_rep2_count': 'D05_R2',
                              'D05_rep3_count': 'D05_R3',
                              'D13_rep1_count': 'D13_R1',
                              'D13_rep2_count': 'D13_R2',
                              'D13_rep3_count': 'D13_R3'
                             }
                  )


    df = df.drop(columns = ['D05_rep1_freq', 'D05_rep2_freq', 'D05_rep3_freq',
                            'D13_rep1_freq', 'D13_rep2_freq', 'D13_rep3_freq',
                            'RNA_D05_rep1_count', 'RNA_D05_rep2_count', 'RNA_D05_rep3_count',
                            'RNA_D05_rep1_freq', 'RNA_D05_rep2_freq', 'RNA_D05_rep3_freq'
                           ]
                )


    value_cols = [col for col in df.columns if "D13"  in col or "D05"  in col or "lib_count" in col]
    dfmelt = pd.melt(df, id_vars=['start_pos_id'], value_vars=value_cols)
    dfpivot = pd.pivot(dfmelt, index='variable', columns=['start_pos_id'], values='value')
    dfpivot = dfpivot.rename_axis(None, axis=1)
    dfpivot = dfpivot.rename_axis(None, axis=0)
    
    metanames = ['D05_R1', 'D05_R2', 'D05_R3', 'D13_R1', 'D13_R2', 'D13_R3', 'lib_count']
    metadays = [5, 5, 5, 13, 13, 13, 0]

    metadf = pd.DataFrame(
        {'sample_name': metanames,
        'time': metadays,
        }
    )
    metadf = metadf.set_index('sample_name').rename_axis(None, axis=0)

    inference = DefaultInference(n_cpus=1)
    dds = DeseqDataSet(
        counts=dfpivot,
        metadata=metadf,
        design="~time",
        refit_cooks=True
    )
    
    dds.deseq2()
    contrast = ["time", 1, 0]
    stat_res = DeseqStats(dds, contrast=contrast, inference=inference)
    stat_res.summary()
    resdf = stat_res.results_df

    resdf = resdf.reset_index(names='start_pos_id')#.drop(columns=['index'])
    resdf = resdf.merge(df[[
        "pos", "allele", "start_pos_id", 'AAsub', 'Consequence', 'canonical_start', 'second_start'
    ]])
    resdf["target"] = 'BARD1_X1A_ATG'

    resdf = resdf[['start_pos_id', 'log2FoldChange', 'lfcSE', 'pos', 'allele', 'AAsub', 'Consequence', 'canonical_start', 'second_start', 'target']]
    resdf = resdf.rename(columns = {'log2FoldChange': 'score', 'lfcSE': 'standard_error'})

    resdf = resdf.loc[:, ['target', 'pos', 'allele', 'start_pos_id', 'Consequence', 'AAsub', 'canonical_start', 'second_start', 'score', 'standard_error']]

    def starts(df):
        df['comb_starts'] = df['canonical_start'] + df['second_start']

        df['start_consequence'] = None

        df.loc[df['comb_starts'].str.match(r'^.{3}ATG$'), 'start_consequence'] = 'canonical_lost'
        df.loc[df['comb_starts'].str.match(r'^ATG.{3}$'), 'start_consequence'] = 'second_lost'
        df.loc[df['comb_starts'] == 'ATGATG', 'start_consequence'] = 'WT'
        df.loc[df['start_consequence'].isna(), 'start_consequence'] = 'both_lost'

        #df = df.drop(columns = ['comb_starts'])
        
        return df

    resdf = starts(resdf)

    resdf = pd.merge(resdf, original_df, on = 'start_pos_id', how = 'left')
    #resdf.to_excel('../Data/ATG_lib_data/20250409_BARD1_X1A_ATG_scored.xlsx', index = False)

    return resdf

In [ ]:
def visualize_scores(df): #Visualizes DNA scores

    ordered_var_list = ['synonymous_variant', 'missense_variant', 'stop_gained']
    facet_sort = ['WT', 'canonical_lost', 'second_lost', 'both_lost']
    histogram = alt.Chart(df).mark_bar().encode(
        x = alt.X('score', title = 'SGE Score', bin = alt.Bin(maxbins = 40)),
        y = alt.Y('count()', title = 'Number of Variants'),
        color = alt.Color('Consequence', sort = ordered_var_list)
    ).properties(
        width = 600, 
        height = 400
    ).interactive()

    facetgram = histogram.facet(columns = 2, facet = alt.Facet('start_consequence', title = 'Start Codon Impact', sort = facet_sort))

    facetgram.display()

    scattergram = alt.Chart(df).mark_point().encode(
        x = alt.X('pos:O', title = 'Genomic Coordinate', axis = alt.Axis(labels = False)),
        y = alt.Y('score', title = 'SGE Score'),
        color = alt.Color('Consequence',sort = ordered_var_list),
        tooltip = [alt.Tooltip('AAsub', title = 'AA Substitution: ')]
    ).properties(
        width = 600,
        height = 400
    ).facet(
        columns = 2,
        facet = alt.Facet('start_consequence',
                          title = 'Start Codon Impact',
                          sort = facet_sort
                         )
    )

    scattergram.display()

    std_error = alt.Chart(df).mark_point().encode(
        x = 'pos:O',
        y = 'standard_error',
        color = 'start_consequence'
    )

    std_error.display()

In [ ]:
def main():
    counts_dict = read_counts(counts_path)
    freq_dicts, keys = add_freq(counts_dict)
    mapped_snvs = annotate_snv_lib()
    annotated = annotate_vars(freq_dicts, keys, mapped_snvs, x1a_annotations)
    print(mapped_snvs)
    combined_df = combine_dfs(annotated, keys)
    qc_plots(combined_df)
    scored_df = dna_score(combined_df)
    visualize_scores(scored_df)

In [ ]:
main()